<a href="https://colab.research.google.com/github/NataliaSchwachulova/KJ_prezencka/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive

In [9]:
drive.mount("/content/drive", force_remount=True)
root_dir = "/content/drive/MyDrive/"
base_dir = root_dir + "MPC-VMM/"

data_path = base_dir + "project/"

Mounted at /content/drive


In [36]:
df = pd.read_csv(data_path + 'red_wine_quality_Final.csv')
df.head()

,fixedAcidity,volatileAcidity,citricAcid,residualSugar,chlorides,freeSulfurDioxide,totalSulfurDioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,NaN,3.51,0.56,9.4,5


In [37]:
print(df['volatileAcidity'].min(), df['volatileAcidity'].max())

0.12 1.58


In [38]:
# odstranění NAN (a inf)
df = pd.read_csv(data_path + 'red_wine_quality_Final.csv')
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df = df.dropna()

df.head()
df

,fixedAcidity,volatileAcidity,citricAcid,residualSugar,chlorides,freeSulfurDioxide,totalSulfurDioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
5,7.4,0.660,0.00,1.8,0.075,13.0,40.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1592,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,-1.00,0.75,11.0,6
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [39]:
#odstranění pH pod 0 a nad 14
df = df.drop(df[df['pH'] < 0].index)
df = df.drop(df[df['pH'] > 14].index)


In [41]:
# MULTIKOLINERAITA
from statsmodels.stats.outliers_influence import variance_inflation_factor
from colorama import Back, Fore

VIFs = [variance_inflation_factor(df.values, i) for i in range (df.shape[1] - 1)]

for v in VIFs:
    print(Fore.LIGHTMAGENTA_EX, end='')
    if v > 10: print(Fore.RED, end='')
    if v < 5: print(Fore.BLACK, end='')
    print("%-8s" % df.columns[VIFs.index(v)] + ": %.1f" % v, Back.RESET)




fixedAcidity: 74.8 
volatileAcidity: 18.1 
citricAcid: 9.5 
residualSugar: 5.1 
chlorides: 7.2 
freeSulfurDioxide: 6.4 
totalSulfurDioxide: 6.6 
density : 1558.3 
pH      : 1096.1 
sulphates: 20.9 
alcohol : 148.0 


In [67]:
# logistická regrese na všech parametrech
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X= df.drop(['quality'],axis=1)
y = df['quality']

X_train, X_test,y_train, y_test = train_test_split(X,y,test_size=0.30, random_state=42)


logreg = LogisticRegression(max_iter=10000)
logreg.fit(X_train,y_train)
y_pred = logreg.predict(X_test)
print(logreg.score(X_train, y_train))
print(logreg.score(X_test, y_test))



0.6179624664879356
0.5700934579439252


In [48]:
df2 = df.drop('density', axis=1)

VIFs = [variance_inflation_factor(df2.values, i) for i in range (df2.shape[1] - 1)]

for v in VIFs:
    print(Fore.LIGHTMAGENTA_EX, end='')
    if v > 10: print(Fore.RED, end='')
    if v < 5: print(Fore.BLACK, end='')
    print("%-8s" % df2.columns[VIFs.index(v)] + ": %.1f" % v, Back.RESET)

fixedAcidity: 42.8 
volatileAcidity: 18.1 
citricAcid: 9.5 
residualSugar: 5.1 
chlorides: 6.6 
freeSulfurDioxide: 6.3 
totalSulfurDioxide: 6.1 
pH      : 164.3 
sulphates: 20.9 
alcohol : 146.9 


In [70]:
X = df2.drop(['quality'],axis=1)
y = df2['quality']

X_train, X_test,y_train, y_test = train_test_split(X,y,test_size=0.30, random_state=42)


logreg = LogisticRegression(max_iter=10000)
logreg.fit(X_train,y_train)
y_pred = logreg.predict(X_test)
print(logreg.score(X_train, y_train))
print(logreg.score(X_test, y_test))

0.6179624664879356
0.5669781931464174


In [49]:
df3 = df2.drop('alcohol', axis=1)

VIFs = [variance_inflation_factor(df3.values, i) for i in range (df3.shape[1] - 1)]

for v in VIFs:
    print(Fore.LIGHTMAGENTA_EX, end='')
    if v > 10: print(Fore.RED, end='')
    if v < 5: print(Fore.BLACK, end='')
    print("%-8s" % df3.columns[VIFs.index(v)] + ": %.1f" % v, Back.RESET)

fixedAcidity: 42.5 
volatileAcidity: 18.0 
citricAcid: 9.1 
residualSugar: 5.0 
chlorides: 6.3 
freeSulfurDioxide: 6.3 
totalSulfurDioxide: 6.0 
pH      : 97.8 
sulphates: 20.8 


In [73]:
X= df3.drop(['quality'],axis=1)
y = df3['quality']

X_train, X_test,y_train, y_test = train_test_split(X,y,test_size=0.30, random_state=42)


logreg = LogisticRegression(max_iter=10000)
logreg.fit(X_train,y_train)
y_pred = logreg.predict(X_test)
print(logreg.score(X_train, y_train))
print(logreg.score(X_test, y_test))

0.579088471849866
0.5077881619937694


In [50]:
df4 = df3.drop('pH', axis=1)

VIFs = [variance_inflation_factor(df4.values, i) for i in range (df4.shape[1] - 1)]

for v in VIFs:
    print(Fore.LIGHTMAGENTA_EX, end='')
    if v > 10: print(Fore.RED, end='')
    if v < 5: print(Fore.BLACK, end='')
    print("%-8s" % df4.columns[VIFs.index(v)] + ": %.1f" % v, Back.RESET)

fixedAcidity: 40.7 
volatileAcidity: 13.9 
citricAcid: 8.9 
residualSugar: 5.0 
chlorides: 6.3 
freeSulfurDioxide: 6.3 
totalSulfurDioxide: 5.9 
sulphates: 20.2 


In [74]:
X = df4.drop(['quality'],axis=1)
y = df4['quality']

X_train, X_test,y_train, y_test = train_test_split(X,y,test_size=0.30, random_state=42)


logreg = LogisticRegression(max_iter=10000)
logreg.fit(X_train,y_train)
y_pred = logreg.predict(X_test)
print(logreg.score(X_train, y_train))
print(logreg.score(X_test, y_test))

0.5683646112600537
0.5015576323987538


In [51]:
df5 = df4.drop('fixedAcidity', axis=1)

VIFs = [variance_inflation_factor(df5.values, i) for i in range (df5.shape[1] - 1)]

for v in VIFs:
    print(Fore.LIGHTMAGENTA_EX, end='')
    if v > 10: print(Fore.RED, end='')
    if v < 5: print(Fore.BLACK, end='')
    print("%-8s" % df5.columns[VIFs.index(v)] + ": %.1f" % v, Back.RESET)

volatileAcidity: 10.3 
citricAcid: 4.8 
residualSugar: 4.9 
chlorides: 6.3 
freeSulfurDioxide: 6.2 
totalSulfurDioxide: 5.9 
sulphates: 20.0 


In [53]:
df6 = df5.drop('sulphates', axis=1)

VIFs = [variance_inflation_factor(df6.values, i) for i in range (df6.shape[1] - 1)]

for v in VIFs:
    print(Fore.LIGHTMAGENTA_EX, end='')
    if v > 10: print(Fore.RED, end='')
    if v < 5: print(Fore.BLACK, end='')
    print("%-8s" % df6.columns[VIFs.index(v)] + ": %.1f" % v, Back.RESET)

volatileAcidity: 10.3 
citricAcid: 4.7 
residualSugar: 4.9 
chlorides: 5.5 
freeSulfurDioxide: 6.2 
totalSulfurDioxide: 5.8 
